In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import gridspec
import time

In [2]:
from matplotlib import rc
rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

mpl.rcParams.update({'font.size': 19})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

import matplotlib.pyplot as plt
import mplhep as hep
hep.set_style("CMS") 

<ipython-input-2-99a23e5daf14>:21: FutureWarning: ``set_style`` is deprecated: Naming convention is changing to match mpl. Use ``mplhep.style.use()``.
  hep.set_style("CMS")


### Get spectrum for Q^2 and Q^2_zoomed

In [5]:
qedcorr_rapgap = {}
qedcorr_djangoh = {} 

with open('outputfiles/QEDCorrections_Rapgap.npy' , 'rb') as f:
    qedcorr_rapgap['pT'] = np.load(f)[0:4]
    qedcorr_rapgap['eta'] = np.load(f)
    qedcorr_rapgap['qT'] = np.load(f)
    qedcorr_rapgap['dphi'] = np.load(f)
with open('outputfiles/QEDCorrections_Djangoh.npy' , 'rb') as f:
    qedcorr_djangoh['pT'] = np.load(f)[0:4]
    qedcorr_djangoh['eta'] = np.load(f)
    qedcorr_djangoh['qT'] = np.load(f)
    qedcorr_djangoh['dphi'] = np.load(f)
    

mc = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Django_nominal.pkl")
theta0_G = mc[['genjet_pt','genjet_eta','genjet_dphi','genjet_qtnorm','gen_Q2']].to_numpy()
weights_MC = mc['wgt']
pass_fid = np.array(mc['pass_fiducial'])
Q2 = theta0_G[:,4][pass_fid==1]

del mc
gc.collect()


file_loc = "storage_files_old_pT_bin"


symbol = {}


symbol['Q2'] = '$Q^{2}\ [GeV^2]$'
symbol['Q2_zoomed'] = '$Q^{2}\ [GeV^2]$'

bins={}
bins['Q2'] = np.load("/global/home/users/yxu2/disjets/FinalReading/Qbinned_result/inputfiles/Q2_full_bin.npy")
bins['Q2_zoomed'] = np.logspace(np.log10(100),np.log10(1000),20)



truthval = {}
truthval["Q2"],_=np.histogram(theta0_G[pass_fid==1][:,4],bins=bins['Q2'],weights=weights_MC[pass_fid==1],density=True)
truthval["Q2_zoomed"],_=np.histogram(theta0_G[pass_fid==1][:,4],bins=bins['Q2_zoomed'],weights=weights_MC[pass_fid==1],density=True)


for obs in ['Q2','Q2_zoomed']:

    nominal_R = []
    sys0_R = []
    sys1_R = []
    sys5_R = []
    sys7_R = []
    sys11_R = []

    nominal_D = []
    sys0_D = []
    sys1_D = []
    sys5_D = []
    sys7_D = []
    sys11_D = []


    for i in range(8):

        nominal_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4.npy")]
        sys0_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_0n_Omni_step2_"+obs+"_iteration4.npy")]
        sys1_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_1n_Omni_step2_"+obs+"_iteration4.npy")]
        sys5_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_5n_Omni_step2_"+obs+"_iteration4.npy")]
        sys7_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_7n_Omni_step2_"+obs+"_iteration4.npy")]
        sys11_R += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Rapgap_sys_11n_Omni_step2_"+obs+"_iteration4.npy")]

        nominal_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_nominaln_Omni_step2_"+obs+"_iteration4.npy")]
        sys0_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_0n_Omni_step2_"+obs+"_iteration4.npy")]
        sys1_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_1n_Omni_step2_"+obs+"_iteration4.npy")]
        sys5_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_5n_Omni_step2_"+obs+"_iteration4.npy")]
        sys7_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_7n_Omni_step2_"+obs+"_iteration4.npy")]
        sys11_D += [np.load("inputfiles/fullscan"+str(i+1)+"/"+file_loc+"/Django_sys_11n_Omni_step2_"+obs+"_iteration4.npy")]
        pass

    #non-closure
    nc = np.load("inputfiles/nonclosure/"+file_loc+"/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4_NC_.npy")
    nc = (nc-truthval[obs])/truthval[obs]

    #stat uncert
    nominal_stat = []
    for seed in np.concatenate([range(1,30),range(34,45),[46,47,48,49],range(54,66),range(80,86),range(100,106),range(120,126)]):
        nominal_stat += [np.load("inputfiles/fullscan_stat/"+file_loc+"/Rapgap_nominaln_Omni_step2_"+obs+"_iteration4_"+str(seed)+".npy")]
        pass

    nominal_stat = nominal_stat[0:74]
    nominal_stat = np.reshape(nominal_stat,[74,1,np.shape(nominal_stat)[-1]])
    nominal_stat = np.median(nominal_stat,axis=1)


    fig = plt.figure()

    nominal_rap = np.median(nominal_R,axis=0)

    sys0_rap = np.median(sys0_R,axis=0)
    sys1_rap = np.median(sys1_R,axis=0)
    sys5_rap = np.median(sys5_R,axis=0)
    sys7_rap = np.median(sys7_R,axis=0)
    sys11_rap = np.median(sys11_R,axis=0)

    #std
    std_nom_R = np.std(nominal_R,axis=0)/np.median(nominal_R,axis=0)
    std_sys0_R = np.std(sys0_R,axis=0)/np.median(sys0_R,axis=0)
    std_sys1_R = np.std(sys1_R,axis=0)/np.median(sys1_R,axis=0)
    std_sys5_R = np.std(sys5_R,axis=0)/np.median(sys5_R,axis=0)
    std_sys7_R = np.std(sys7_R,axis=0)/np.median(sys7_R,axis=0)
    std_sys11_R = np.std(sys11_R,axis=0)/np.median(sys11_R,axis=0)
    std_nom_D = np.std(nominal_D,axis=0)/np.median(nominal_D,axis=0)
    std_sys0_D = np.std(sys0_D,axis=0)/np.median(sys0_D,axis=0)
    std_sys1_D = np.std(sys1_D,axis=0)/np.median(sys1_D,axis=0)
    std_sys5_D = np.std(sys5_D,axis=0)/np.median(sys5_D,axis=0)
    std_sys7_D = np.std(sys7_D,axis=0)/np.median(sys7_D,axis=0)
    std_sys11_D = np.std(sys11_D,axis=0)/np.median(sys11_D,axis=0)
    std = np.mean([std_nom_R,std_sys0_R,std_sys1_R,std_sys5_R,std_sys7_R,std_sys11_R,
                  std_nom_D,std_sys0_D,std_sys1_D,std_sys5_D,std_sys7_D,std_sys11_D],axis=0)
    stat = np.std(nominal_stat,axis=0)/np.mean(nominal_stat,axis=0)
    stat = np.sqrt(abs(stat**2-std**2))



    nominal_dj = np.median(nominal_D,axis=0)
    sys0_dj = np.median(sys0_D,axis=0)
    sys1_dj = np.median(sys1_D,axis=0)
    sys5_dj = np.median(sys5_D,axis=0)
    sys7_dj = np.median(sys7_D,axis=0)
    sys11_dj = np.median(sys11_D,axis=0)

    myavgR_0 = (nominal_rap - sys0_rap)/nominal_rap
    myavgR_1 = (nominal_rap - sys1_rap)/nominal_rap
    myavgR_5 = (nominal_rap - sys5_rap)/nominal_rap
    myavgR_7 = (nominal_rap - sys7_rap)/nominal_rap
    myavgR_11 = (nominal_rap - sys11_rap)/nominal_rap

    myavgD_0 = (nominal_dj - sys0_dj)/nominal_dj
    myavgD_1 = (nominal_dj - sys1_dj)/nominal_dj
    myavgD_5 = (nominal_dj - sys5_dj)/nominal_dj
    myavgD_7 = (nominal_dj - sys7_dj)/nominal_dj
    myavgD_11 = (nominal_dj - sys11_dj)/nominal_dj

    myavg_0 = np.mean([myavgR_0,myavgD_0],axis=0)
    myavg_1 = np.mean([myavgR_1,myavgD_1],axis=0)
    myavg_5 = np.mean([myavgR_5,myavgD_5],axis=0)
    myavg_7 = np.mean([myavgR_7,myavgD_7],axis=0)
    myavg_11 = np.mean([myavgR_11,myavgD_11],axis=0)

    model_nominal = (nominal_rap-nominal_dj)/nominal_rap
    model_0 = (sys0_rap-sys0_dj)/sys0_rap
    model_1 = (sys1_rap-sys1_dj)/sys1_rap
    model_5 = (sys5_rap-sys5_dj)/sys5_rap
    model_7 = (sys7_rap-sys7_dj)/sys7_rap
    model_11 = (sys11_rap-sys11_dj)/sys11_rap
    model = np.mean([model_nominal,model_0,model_1,model_5,model_7,model_11],axis=0)
    #qed = (qedcorr_rapgap[obs]-qedcorr_djangoh[obs])/qedcorr_rapgap[obs]

    fig = plt.figure(figsize=(7, 5)) 
    gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    xvals = 0.5*(bins[obs][0:-1]+bins[obs][1:])

    #plt.plot(xvals, 100*abs(qed),label="QED rad. corr.",color="tab:blue")
    plt.plot(xvals, 100*abs(nc),label="Non-closure",color="tab:orange")
    plt.plot(xvals,100*abs(myavg_0),label="HFS scale (in jet)",color="tab:green")
    plt.plot(xvals,100*abs(myavg_1),label="HFS scale (remainder)",color="tab:red")
    plt.plot(xvals,100*abs(myavg_5),label="HFS $\phi$ angle",color="tab:purple")
    plt.plot(xvals,100*abs(myavg_7),label="Lepton energy scale",color="tab:brown")
    plt.plot(xvals,100*abs(myavg_11),label="Lepton $\phi$ angle",color="tab:pink")
    plt.plot(xvals,100*abs(model),label="Model",color="tab:gray")
    plt.plot(xvals,100*abs(stat),label="Stat.",ls="--",lw=3,color="tab:olive")

    #total = np.sqrt(nc**2+qed**2+myavg_0**2+myavg_1**2+myavg_5**2+myavg_7**2+myavg_11**2+model**2)
    #turned off QED uncertainty
    total = np.sqrt(nc**2+myavg_0**2+myavg_1**2+myavg_5**2+myavg_7**2+myavg_11**2+model**2)
    plt.plot(xvals,100*abs(total),ls=":",color="black",label="Total Syst.",lw=3)


    plt.legend(loc='upper right',fontsize=14,ncol=2,bbox_to_anchor=(0.96, 0.9))
    plt.ylim([0,32])
    plt.xlabel(symbol[obs],fontsize=20)
    plt.ylabel("Systematic Uncertainty [%]",fontsize=20)


    plt.xscale("log")


    plt.text(0.1, 1.15-0.25,'H1', horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes, fontsize=25, fontweight='bold')
    plt.text(0.6, 1.15-0.25, '$0.2<y<0.7$, $p_{T}^{jet}>10$ GeV', horizontalalignment='center',verticalalignment='center', transform = ax0.transAxes, fontsize=13)

    myobs_map = {}
    myobs_map['Q2'] = 'Q2'
    myobs_map['Q2_zoomed'] = 'Q2_zoomed'


    fig.savefig('figures/TotalUncerts_%s_FIXED.png'%myobs_map[obs],bbox_inches='tight')

    #save results
    with open('outputfiles/ResultwithSystematicUncertainties_%s_ensemble_FIXED.npy'%myobs_map[obs], 'wb') as f:
        np.save(f,0.5*(bins[obs][0:-1]+bins[obs][1:]))
        np.save(f,np.mean([nominal_rap,nominal_dj],axis=0))
        np.save(f,0.5*abs(bins[obs][0:-1]-bins[obs][1:]))
        np.save(f, total)
        np.save(f, stat)
        #np.save(f,abs(qed))
        np.save(f,abs(myavg_0))
        np.save(f,abs(myavg_1))
        np.save(f,abs(myavg_5))
        np.save(f,abs(myavg_7))
        np.save(f,abs(myavg_11))
        np.save(f,abs(model))

del theta0_G,weights_MC,pass_fid
gc.collect()

<ipython-input-5-32d284ebc5f6>:82: RuntimeWarning: invalid value encountered in true_divide
  nc = (nc-truthval[obs])/truthval[obs]
<ipython-input-5-32d284ebc5f6>:106: RuntimeWarning: invalid value encountered in true_divide
  std_nom_R = np.std(nominal_R,axis=0)/np.median(nominal_R,axis=0)
<ipython-input-5-32d284ebc5f6>:107: RuntimeWarning: invalid value encountered in true_divide
  std_sys0_R = np.std(sys0_R,axis=0)/np.median(sys0_R,axis=0)
<ipython-input-5-32d284ebc5f6>:108: RuntimeWarning: invalid value encountered in true_divide
  std_sys1_R = np.std(sys1_R,axis=0)/np.median(sys1_R,axis=0)
<ipython-input-5-32d284ebc5f6>:109: RuntimeWarning: invalid value encountered in true_divide
  std_sys5_R = np.std(sys5_R,axis=0)/np.median(sys5_R,axis=0)
<ipython-input-5-32d284ebc5f6>:110: RuntimeWarning: invalid value encountered in true_divide
  std_sys7_R = np.std(sys7_R,axis=0)/np.median(sys7_R,axis=0)
<ipython-input-5-32d284ebc5f6>:111: RuntimeWarning: invalid value encountered in true_di

10114